In [112]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import pims
import pickle
from skimage.measure import regionprops
from pathlib import Path
from gzip import GzipFile

In [7]:
home_path = Path('/home/idies/workspace/codex/PD1_polarization/')
cell_data = pd.read_csv(home_path / 'reg5_mouse89_outlines_and_expression.csv')

In [8]:
channelNames = list(pd.read_csv(home_path / 'channelNames.txt',header = None)[0].values)

In [9]:
cell_data.head()

,CellID,tilename,Xroi,Yroi,Xtile,Ytile,ClusterID,CD62L,Siglec-H,CD86,...,X_withinTile,Y_withinTile,Z,size,cyc20_ch2blnk,cyc20_ch3blnk,cyc20_ch4blnk,cyc21_ch2blnk,cyc21_ch3blnk,cyc21_ch4blnk
0,0-1201-4,reg005_X01_Y02,2_2_2_2_2_2_2_2_2_2_2_1_1_1_1_1_1_1_1_1_1_1_0_...,478_479_480_481_482_483_484_485_486_487_488_47...,1,2,3262,477.369232,46.163620,123.079987,...,0,481,4,566,0.086064,0.000000,0.000000,0.511932,0.000000,0.000000
1,0-1216-4,reg005_X01_Y02,2_2_2_2_2_2_2_2_2_2_2_2_2_2_1_2_1_1_1_1_1_1_1_...,489_490_491_492_493_494_495_496_497_498_499_50...,1,2,3262,0.000000,0.000000,117.690712,...,0,496,4,585,0.000000,3.202495,4.067975,0.000000,12.229112,21.761768
2,0-123-4,reg005_X01_Y01,0_0_0_0_0_1_0_1_0_1_0_1_0_1_0_1_0_1_0_1_0_1_0_...,104_105_106_107_108_104_109_105_110_106_111_10...,1,1,3223,0.000000,0.000000,0.000000,...,0,123,4,4086,36.213600,5.332117,1.150246,90.962128,16.062859,4.204434
3,0-1268-3,reg005_X01_Y03,0_0_0_0_0_0_0_0_0_0_0_1_0_0_1_0_1_0_0_1_0_1_0_...,26_27_28_29_30_31_32_33_34_35_36_26_37_38_27_3...,1,3,3262,383.789124,62.661911,265.210876,...,0,44,3,2519,0.000000,2.596656,4.500194,0.000000,11.984420,16.527153
4,0-1285-3,reg005_X01_Y03,0_0_0_0_0_0_0_0_0_1_1_1_1_1_1_1_1_1_1_2_2_2_2_...,57_58_59_60_62_63_64_65_66_58_59_60_61_62_63_6...,1,3,3246,0.000000,0.000000,8.862731,...,0,61,3,385,1.368026,0.000000,0.000000,3.603063,0.000000,0.000000


In [18]:
def get_channel_index(chan,num_zs=7,z_indices = range(7)):
    cycle = chan//4
    channel = chan%4
    index = [cycle*4*num_zs+(z*4)+channel for z in z_indices]
    return index  

    
    
def read_cells(tile_name,cell_data):
    '''
    extract cells of given tile.  
    
    tile_name:  string of tile name (not including '.tif' e.g: reg005_X01_Y01)
    cell_data:  dataframe of cell regions, expression info and other meta data
    
    returns subset of cell_data corresponding to a tile and dict with keys as CellID and values as np array of region pixels 
    *note values are in x,y format (not rows, cols)
    '''
    tile_cells = cell_data[cell_data['tilename']==tile_name]
    roisX = np.core.defchararray.split(tile_cells['Xroi'].values.astype(str),sep = '_')
    roisY = np.core.defchararray.split(tile_cells['Yroi'].values.astype(str),sep = '_')
    roi = {i:np.array([x,y]).astype(int) for (i,x,y) in zip(tile_cells['CellID'].values,roisX,roisY) if (len(x)>1 and len(y)>1)} 
    return tile_cells,roi

def read_tile_image(tile_name,channelNames,markers,zs,tile_direc ):
    '''
    extract desired channels and zs from raw image
    
    tile_name:  string of tile name (not including '.tif' e.g: reg005_X01_Y01)
    channelNames:  list of proteins in order found in image (not including Zs)
    markers:    list of proteins to view  e.g: PD1,CD3 etc 
    zs:         list of z coordinates to extract
    tile_direc:  directory where tile data is saved
    
    *note raw image is flattened from following shape (num_cycles,num_zs,num_channels)
    
    returns: numpy array in following shape (num_markers,num_zs,nrows,ncols)
    '''
    indices = [get_channel_index(channelNames.index(marker),z_indices = zs) for marker in markers]
    indices = [k for i in indices for k in i]
    path = tile_direc / '{}.tif'.format(tile_name)
    s = pims.TiffStack(path)
    img = np.array(s[indices]).reshape(len(markers),len(zs),*s.frame_shape)
    return img


def match_tile_and_cells(tile_name,cell_data,channelNames,markers,tile_direc = home_path / 'mouse_number_89_reg005_tiles'):
    '''
    wrapper function to get imge and cells of a given tile with desired markers and zs 
    
    tile_name:  string of tile name (not including '.tif' e.g: reg005_X01_Y01)
    cell_data:  dataframe of cell regions, expression info and other meta data
    channelNames:  list of proteins in order found in image (not including Zs)
    markers:    list of proteins to view  e.g: PD1,CD3 etc
    tile_direc:  directory where tile data is saved
    
    returns: np.array(num_markers,num_zs,nrows,ncols),dataframe (subset of cell_data), dict cellid: cell pixels (x,y))
    
    '''
    img = read_tile_image(tile_name,channelNames,markers,zs,tile_direc)
    tile_cells,roi = read_cells(tile_name,cell_data)
    return img,tile_cells,roi
    
    
    
    

## Cell Tensors


In [21]:
tile_names = [i.split('.tif')[0] for i in os.listdir(home_path / 'mouse_number_89_reg005_tiles') if 'tif' in i]
tile_names

['reg005_X01_Y02',
 'reg005_X05_Y02',
 'reg005_X05_Y05',
 'reg005_X01_Y05',
 'reg005_X06_Y07',
 'reg005_X02_Y07',
 'reg005_X06_Y09',
 'reg005_X02_Y09',
 'reg005_X07_Y08',
 'reg005_X03_Y08',
 'reg005_X07_Y06',
 'reg005_X03_Y06',
 'reg005_X03_Y01',
 'reg005_X07_Y01',
 'reg005_X04_Y03',
 'reg005_X04_Y04',
 'reg005_X06_Y08',
 'reg005_X02_Y08',
 'reg005_X02_Y01',
 'reg005_X06_Y01',
 'reg005_X06_Y06',
 'reg005_X02_Y06',
 'reg005_X05_Y04',
 'reg005_X01_Y04',
 'reg005_X01_Y03',
 'reg005_X05_Y03',
 'reg005_X04_Y05',
 'reg005_X04_Y02',
 'reg005_X07_Y07',
 'reg005_X03_Y07',
 'reg005_X07_Y09',
 'reg005_X03_Y09',
 'reg005_X04_Y01',
 'reg005_X04_Y06',
 'reg005_X04_Y08',
 'reg005_X07_Y04',
 'reg005_X03_Y04',
 'reg005_X03_Y03',
 'reg005_X07_Y03',
 'reg005_X06_Y05',
 'reg005_X02_Y05',
 'reg005_X02_Y02',
 'reg005_X06_Y02',
 'reg005_X05_Y09',
 'reg005_X01_Y09',
 'reg005_X05_Y07',
 'reg005_X01_Y07',
 'reg005_X03_Y02',
 'reg005_X07_Y02',
 'reg005_X07_Y05',
 'reg005_X03_Y05',
 'reg005_X04_Y09',
 'reg005_X04

In [107]:
save_dir = Path('/home/idies/workspace/Temporary/chug/scratch/')
with open('/home/idies/workspace/Storage/bmyury/shared_scripts_and_files/segmentation_masks.pkl', 'rb') as f:
    segmentation_masks = pickle.load(f)

save_dir.mkdir(exist_ok=True)
channelNames = list(pd.read_csv(home_path / 'channelNames.txt', header = None)[0].values)
cell_data = pd.read_csv(home_path /'reg5_mouse89_outlines_and_expression.csv')

gated_positive = pd.read_csv(home_path / 'PD1 positive cells of reg5 mouse 89 gated using TYRP1_PD1.txt')
gated_tile = gated_positive[gated_positive['tilename']==tile_name]


In [38]:
plt.imshow(segmentation_masks[('reg005_X01_Y06', 3)])

In [39]:
img = read_tile_image('reg005_X01_Y06',channelNames,channelNames[:3],[4, 5], home_path / 'mouse_number_89_reg005_tiles')

In [41]:
img.shape

(3, 2, 504, 672)

In [43]:
%pdb

Automatic pdb calling has been turned ON


In [96]:
def make_cell_stack(
    cell_data, channelNames, markers, segmentation_masks,
    mask_cells=True, box_size=32, tile_path=home_path / 'mouse_number_89_reg005_tiles'
):
    cell_data["cell_ix"] = np.arange(cell_data.shape[0]) + 1
    cell_stack = np.empty((cell_data.shape[0], len(markers), box_size, box_size), dtype=np.uint16)
    skipped_cells = []
    cd_grouped = cell_data.groupby(["Z", "tilename"])
    for (z, tn), cd in cd_grouped:
        print((tn, z))
        img = read_tile_image(tn, channelNames, markers, [z], tile_path)
        if mask_cells:
            img[:, :, segmentation_masks[(tn, z)] < 1] = 0
        props = regionprops(segmentation_masks[(tn, z)])
        centroid_df = pd.DataFrame({"cell_ix": [x.label for x in props], "centroid": [x.centroid for x in props]})
        cd = cd.merge(centroid_df, how="left", on="cell_ix")
        for c in cd.itertuples():
            centr = c.centroid
            ci = img[
                :,
                0,
                (int(centr[0]) - box_size//2) : (int(centr[0]) + box_size//2),
                (int(centr[1]) - box_size//2) : (int(centr[1]) + box_size//2)
            ]
            if ci.shape == (len(markers), box_size, box_size):
                cell_stack[c.cell_ix - 1, :, :, :] = ci
            else:
                skipped_cells.append(c.cell_ix)
    return cell_data[~cell_data.cell_ix.isin(skipped_cells)], np.delete(cell_stack, np.array(skipped_cells) - 1, axis=0)

In [93]:
cell_data_stack, cell_stack = make_cell_stack(cell_data, channelNames, channelNames, segmentation_masks)

('reg005_X01_Y06', 0)
('reg005_X01_Y06', 1)
('reg005_X04_Y01', 1)
('reg005_X07_Y07', 1)
('reg005_X07_Y09', 1)
('reg005_X01_Y06', 2)
('reg005_X01_Y09', 2)
('reg005_X02_Y01', 2)
('reg005_X02_Y03', 2)
('reg005_X02_Y06', 2)
('reg005_X03_Y01', 2)
('reg005_X03_Y05', 2)
('reg005_X04_Y05', 2)
('reg005_X04_Y07', 2)
('reg005_X04_Y09', 2)
('reg005_X05_Y04', 2)
('reg005_X05_Y05', 2)
('reg005_X05_Y06', 2)
('reg005_X06_Y05', 2)
('reg005_X06_Y07', 2)
('reg005_X06_Y09', 2)
('reg005_X07_Y01', 2)
('reg005_X07_Y02', 2)
('reg005_X07_Y03', 2)
('reg005_X07_Y04', 2)
('reg005_X07_Y05', 2)
('reg005_X07_Y06', 2)
('reg005_X07_Y09', 2)
('reg005_X01_Y01', 3)
('reg005_X01_Y02', 3)
('reg005_X01_Y03', 3)
('reg005_X01_Y04', 3)
('reg005_X01_Y05', 3)
('reg005_X01_Y06', 3)
('reg005_X01_Y07', 3)
('reg005_X01_Y09', 3)
('reg005_X02_Y01', 3)
('reg005_X02_Y02', 3)
('reg005_X02_Y03', 3)
('reg005_X02_Y05', 3)
('reg005_X02_Y06', 3)
('reg005_X02_Y07', 3)
('reg005_X02_Y08', 3)
('reg005_X02_Y09', 3)
('reg005_X03_Y01', 3)
('reg005_X

In [114]:
np.save(
    save_dir / "all_cells_markers_tensor.npy.gz",
    cell_stack
)

In [111]:
cell_data_stack.to_csv(
    save_dir / "all_cells_markers_data.csv",
    index=False
)